In [290]:
import pandas as pd
import ast
import re

In [291]:
def float_or_false(value):
    try:
        if float(value):
            return True
    except ValueError:
        return False

In [292]:
def mod_false(df):
    for i in range(len(df)):
        if df.loc[i,'modifications'] != '':
            df.loc[i,'is_mod'] = 'True'
        else:
            df.loc[i,'is_mod'] = 'False'

In [293]:
def extract_data(text):
    return re.findall(r"'(.*?)'", text)

In [294]:
df = pd.read_csv('PEP_LIFE_original.csv')

In [295]:
df['half_life_data'] = df['half_life_data'].astype(str).str.replace(r'[\[\]():{}]', '', regex=True)
df['modifications'] = df['modifications'].astype(str).str.replace(r'[\'()]', '', regex=True)
df['modifications'] = df['modifications'].str.rstrip(',')

In [296]:
extracted_data = df['half_life_data'].apply(extract_data)
df_expanded = pd.DataFrame(extracted_data.tolist())
#df_expanded[1]=pd.to_numeric(df_expanded[1], errors='coerce')
df['half_life_data'] = df_expanded[1]

In [297]:
df_concat = pd.concat([df, df_expanded], axis=1)

In [298]:
df_concat.drop(columns=[1], inplace=True)

In [299]:
df_concat

,Unnamed: 0,Unnamed: 1,modifications,references,half_life_data,url_sources,0,2,3,4,5,6
0,RRWQWR,(),,['http://www.ncbi.nlm.nih.gov/pubmed/20844765'],1800,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,Human serum,vivo_vitro,in vitro,http//crdd.osdd.net/raghava/peplife/display_su...
1,RRWWRF,(),,['http://www.ncbi.nlm.nih.gov/pubmed/20844765'],1800,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,Human serum,vivo_vitro,in vitro,http//crdd.osdd.net/raghava/peplife/display_su...
2,RRWQWR,"('C-Terminal Amidation',)",C-Terminal Amidation,['http://www.ncbi.nlm.nih.gov/pubmed/20844765'],1800,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,Human serum,vivo_vitro,in vitro,http//crdd.osdd.net/raghava/peplife/display_su...
3,RRWWRF,"('C-Terminal Amidation',)",C-Terminal Amidation,['http://www.ncbi.nlm.nih.gov/pubmed/20844765'],1800,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,Human serum,vivo_vitro,in vitro,http//crdd.osdd.net/raghava/peplife/display_su...
4,RRWQWR,"('N-Terminal',)",N-Terminal,['http://www.ncbi.nlm.nih.gov/pubmed/20844765'],3600,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,Human serum,vivo_vitro,in vitro,http//crdd.osdd.net/raghava/peplife/display_su...
...,...,...,...,...,...,...,...,...,...,...,...,...
1132,FNAPFDVGIKLSGAQYQQHGRAL,"('C-Terminal Amidation',)",C-Terminal Amidation,['http://www.ncbi.nlm.nih.gov/pubmed/17204551'],1320,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,Intravenous injection of peptide into Sprague...,vivo_vitro,in vivo,http//crdd.osdd.net/raghava/peplife/display_su...
1133,FNAPFDVGIKLSGVQYQQHSQAL,"('C-Terminal Amidation',)",C-Terminal Amidation,['http://www.ncbi.nlm.nih.gov/pubmed/18602197'],3906,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,Mouse plasma,vivo_vitro,in vitro,http//crdd.osdd.net/raghava/peplife/display_su...
1134,FNAPFDVGIKLSGVQYQQHSQAL,"('C-Terminal Amidation', 'Chemical Modificatio...","C-Terminal Amidation, Chemical Modification: I...",['http://www.ncbi.nlm.nih.gov/pubmed/18602197'],906,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,Mouse liver homogenate,vivo_vitro,in vitro,http//crdd.osdd.net/raghava/peplife/display_su...
1135,ASISGRDTHRLTRTLNCSSIVKEIIGKLPEPELKTDDEGPSLRNKS...,"('Cyclic', 'Chemical Modification: Glycosylati...","Cyclic, Chemical Modification: Glycosylation a...",['http://www.ncbi.nlm.nih.gov/pubmed/8353280'],126,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,Injected intravenously in Balb/c mice blood,vivo_vitro,in vivo,http//crdd.osdd.net/raghava/peplife/display_su...


In [300]:
df_concat = df_concat.rename(columns={'Unnamed: 0': 'sequence', 'half_life_data': 'half_life_seconds'})
mod_false(df_concat)

In [301]:
df_concat['hl_is_float'] = df_concat['half_life_seconds'].apply(float_or_false)
df_float=df_concat[df_concat['hl_is_float']==True]
df_object=df_concat[df_concat['hl_is_float']==False]
df_float.reset_index(drop=True, inplace=True)
df_object.reset_index(drop=True, inplace=True)

In [302]:
df_float

,sequence,Unnamed: 1,modifications,references,half_life_seconds,url_sources,0,2,3,4,5,6,is_mod,hl_is_float
0,RRWQWR,(),,['http://www.ncbi.nlm.nih.gov/pubmed/20844765'],1800,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,Human serum,vivo_vitro,in vitro,http//crdd.osdd.net/raghava/peplife/display_su...,False,True
1,RRWWRF,(),,['http://www.ncbi.nlm.nih.gov/pubmed/20844765'],1800,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,Human serum,vivo_vitro,in vitro,http//crdd.osdd.net/raghava/peplife/display_su...,False,True
2,RRWQWR,"('C-Terminal Amidation',)",C-Terminal Amidation,['http://www.ncbi.nlm.nih.gov/pubmed/20844765'],1800,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,Human serum,vivo_vitro,in vitro,http//crdd.osdd.net/raghava/peplife/display_su...,True,True
3,RRWWRF,"('C-Terminal Amidation',)",C-Terminal Amidation,['http://www.ncbi.nlm.nih.gov/pubmed/20844765'],1800,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,Human serum,vivo_vitro,in vitro,http//crdd.osdd.net/raghava/peplife/display_su...,True,True
4,RRWQWR,"('N-Terminal',)",N-Terminal,['http://www.ncbi.nlm.nih.gov/pubmed/20844765'],3600,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,Human serum,vivo_vitro,in vitro,http//crdd.osdd.net/raghava/peplife/display_su...,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1126,FNAPFDVGIKLSGAQYQQHGRAL,"('C-Terminal Amidation',)",C-Terminal Amidation,['http://www.ncbi.nlm.nih.gov/pubmed/17204551'],1320,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,Intravenous injection of peptide into Sprague...,vivo_vitro,in vivo,http//crdd.osdd.net/raghava/peplife/display_su...,True,True
1127,FNAPFDVGIKLSGVQYQQHSQAL,"('C-Terminal Amidation',)",C-Terminal Amidation,['http://www.ncbi.nlm.nih.gov/pubmed/18602197'],3906,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,Mouse plasma,vivo_vitro,in vitro,http//crdd.osdd.net/raghava/peplife/display_su...,True,True
1128,FNAPFDVGIKLSGVQYQQHSQAL,"('C-Terminal Amidation', 'Chemical Modificatio...","C-Terminal Amidation, Chemical Modification: I...",['http://www.ncbi.nlm.nih.gov/pubmed/18602197'],906,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,Mouse liver homogenate,vivo_vitro,in vitro,http//crdd.osdd.net/raghava/peplife/display_su...,True,True
1129,ASISGRDTHRLTRTLNCSSIVKEIIGKLPEPELKTDDEGPSLRNKS...,"('Cyclic', 'Chemical Modification: Glycosylati...","Cyclic, Chemical Modification: Glycosylation a...",['http://www.ncbi.nlm.nih.gov/pubmed/8353280'],126,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,Injected intravenously in Balb/c mice blood,vivo_vitro,in vivo,http//crdd.osdd.net/raghava/peplife/display_su...,True,True


In [303]:
df_object

,sequence,Unnamed: 1,modifications,references,half_life_seconds,url_sources,0,2,3,4,5,6,is_mod,hl_is_float
0,aAGIGILTv,"('Stereochemistry',)",Stereochemistry,['http://www.ncbi.nlm.nih.gov/pubmed/10495424'],Stability increased,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,VMM5 cytotoxic T-lymphocyte line incubated in ...,vivo_vitro,in vitro,http//crdd.osdd.net/raghava/peplife/display_su...,True,False
1,AAGIGILTV,"('N-Terminal',)",N-Terminal,['http://www.ncbi.nlm.nih.gov/pubmed/10495424'],Stability increased,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,VMM5 cytotoxic T-lymphocyte line incubated in ...,vivo_vitro,in vitro,http//crdd.osdd.net/raghava/peplife/display_su...,True,False
2,AAGIGILTV,"('C-Terminal Amidation', 'N-Terminal')","C-Terminal Amidation, N-Terminal",['http://www.ncbi.nlm.nih.gov/pubmed/10495424'],Stability increased,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,VMM5 cytotoxic T-lymphocyte line incubated in ...,vivo_vitro,in vitro,http//crdd.osdd.net/raghava/peplife/display_su...,True,False
3,AAGIGILTV,"('C-Terminal Amidation',)",C-Terminal Amidation,['http://www.ncbi.nlm.nih.gov/pubmed/10495424'],Stability increased,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,VMM5 cytotoxic T-lymphocyte line incubated in ...,vivo_vitro,in vitro,http//crdd.osdd.net/raghava/peplife/display_su...,True,False
4,RPPGFSPFR,(),,['http://www.ncbi.nlm.nih.gov/pubmed/10871321'],Not measured as peptide degraded rapidly30 app...,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,Human blood sample,vivo_vitro,in vivo,http//crdd.osdd.net/raghava/peplife/display_su...,False,False
5,HSEGTFTSDYSKYLDSRRAQDFVQWLMNTKRNRNNIA,(),,['http://www.ncbi.nlm.nih.gov/pubmed/19602537'],Stable,['http://crdd.osdd.net/raghava/peplife/display...,half_life_seconds,test_sample,Mouse plasma Subcutaneous injection,vivo_vitro,in vivo,http//crdd.osdd.net/raghava/peplife/display_su...,False,False


In [304]:
#Por si se quieren quitar las modificaciones
df_filtered = df_float[df_float['is_mod'] == 'False']
df_filtered = df_filtered.drop(columns=['is_mod', 'modifications'])
df_filtered_ob = df_object[df_object['is_mod'] == 'False']
df_filtered_ob = df_filtered_ob.drop(columns=['is_mod', 'modifications'])

In [305]:
df_float=df_float.filter(['sequence', 'half_life_seconds', 'is_mod', 'modifications'])
df_object=df_object.filter(['sequence', 'half_life_seconds', 'is_mod', 'modifications'])

In [306]:
df_float['sequence'] = df_float['sequence'].str.upper()
df_object['sequence'] = df_object['sequence'].str.upper()

In [307]:
df_float.drop_duplicates(subset='sequence', keep='first', inplace=True)
df_object.drop_duplicates(subset='sequence', keep='first', inplace=True)

In [308]:
df_float.dropna(subset=['sequence', 'half_life_seconds'], inplace=True)
df_object.dropna(subset=['sequence', 'half_life_seconds'], inplace=True)

In [309]:
#df_filtered.to_csv('../datasets/PEP_LIFE.csv', index=False)
#df_filtered_ob.to_csv('../datasets/PEP_LIFE.csv', index=False)
df_float.to_csv('PEP_LIFE.csv', index=False)
df_object.to_csv('PEP_LIFE_object.csv', index=False)